In [1]:
!pip3 install owlready2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from owlready2 import *
import datetime

dul_onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()
dul = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")

onto = get_ontology("http://www.semanticweb.org/german_tourism_activities")
onto.imported_ontologies.append(dul_onto)

with onto:
    onto.metadata.comment.append("Authors: Giulio Billi, Cono Cirone")
    onto.metadata.comment.append("Domain: Activity recommendation and tourism in German cities.")
    onto.metadata.comment.append("German City Tourism Activities Ontology")

    class Activity(Thing):
        comment = ["A general concept for an activity of interest. Represents physical locations and events."]
        label = ["Activity"]
        is_a = [dul.Entity]

    class City(Thing):
        comment = ["A German city where an activity is located."]
        label = ["City"]
        is_a = [dul.PhysicalPlace]

    class BudgetTier(Thing):
        comment = ["A classification representing the cost of an Activity (Free, Low, Medium, High)."]
        label = ["Budget Tier"]
        is_a = [dul.Concept]

    class LocationSetting(Thing):
        comment = ["A classification of the environment (Indoor or Outdoor)."]
        label = ["Location Setting"]
        is_a = [dul.Concept]

    class OperatingHours(Thing):
        comment = ["A concept to model a set of opening/closing times."]
        label = ["Operating Hours"]
        is_a = [dul.TimeInterval]

    class DayOfWeek(Thing):
        comment = ["A specific day of the week."]
        label = ["Day of Week"]
        is_a = [dul.Concept]
    
    # # Definisci la classe come un Amount
    class Duration(Thing):
        # La definizione spiega che rappresentiamo quantità temporali
        comment = ["A temporal quantity representing the length of the tour"]
        label = ["Duration"]
        is_a = [dul.Amount]

    class Language(Thing):
        comment = ["A human language in which a tour is offered."]
        label = ["Language"]
        is_a = [dul.InformationObject]

    class MeetingPoint(Thing):
        comment = ["A physical location where a tour starts."]
        label = ["Meeting Point"]
        is_a = [dul.PhysicalPlace]

    class Tour(Activity):
        comment = ["A guided activity to visit the city. It is an event."]
        label = ["Tour"]
        is_a = [dul.Event]

    class PhysicalVenue(Activity):
        comment = ["A physical location with set operating hours."]
        label = ["Physical Venue"]
        is_a = [dul.PhysicalPlace]

    AllDisjoint([Tour, PhysicalVenue])

    class Museum(PhysicalVenue):
        comment = ["An institution displaying artifacts."]
        label = ["Museum"]
    class Park(PhysicalVenue):
        comment = ["A public green space or natural area."]
        label = ["Park"]
    class NightlifeVenue(PhysicalVenue):
        comment = ["A location for evening entertainment."]
        label = ["Nightlife Venue"]
    class Sight(PhysicalVenue):
        comment = ["A location or structure of historical interest."]
        label = ["Sight"]

    AllDisjoint([Museum, Park, NightlifeVenue, Sight])

    class MuseumType(Thing):
        comment = ["Category of museum (e.g., Art, History)."]
        label = ["Museum Type"]
        is_a = [dul.Concept]
    class ParkType(Thing):
        comment = ["Category of park (e.g., Botanical Garden)."]
        label = ["Park Type"]
        is_a = [dul.Concept]
    class ClubType(Thing):
        comment = ["Category of nightlife venue (e.g., Bar, Disco)."]
        label = ["Club Type"]
        is_a = [dul.Concept]
    class SightType(Thing):
        comment = ["Category of sight (e.g., Monument)."]
        label = ["Sight Type"]
        is_a = [dul.Concept]

    AllDisjoint([Activity, City, BudgetTier, LocationSetting, OperatingHours, 
                 DayOfWeek, Duration, Language, MeetingPoint])


    # =========================================================================
    # OBJECT PROPERTIES
    # =========================================================================
    
    # --- Attributes & Classifications ---
    class hasBudget(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Activity]
        range = [BudgetTier]
        label = ["has budget"]
        comment = ["A relation connecting an activity to its cost classification."]
        is_a = [dul.isClassifiedBy] 

    class hasLocationSetting(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Activity]
        range = [LocationSetting]
        label = ["has location setting"]
        comment = ["A relation connecting an activity to its location setting (Indoor vs. Outdoor)."]
        is_a = [dul.isClassifiedBy]

    # --- Spatial & Temporal ---
    
    class isInCity(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Activity]
        range = [City]
        label = ["is in city"]
        comment = ["A relation connecting an activity to the city it is in."]
        is_a = [dul.hasLocation]

    class hasMeetingPoint(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Tour]
        range = [MeetingPoint]
        label = ["has meeting point"]
        comment = ["A relation connecting a tour to its starting location."]
        is_a = [dul.hasLocation]

    
    class hasDuration(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Tour]
        range = [Duration]
        label = ["has duration"]
        comment = ["Relates a tour to its duration amount."]
        is_a = [dul.hasRegion]

    # --- Descriptive & Operational ---

    class hasOperatingHours(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [PhysicalVenue]
        range = [OperatingHours]
        label = ["has operating hours"]
        comment = ["A relation connecting a physical venue to its set of operating hours."]
        # The Venue is described by the OperatingHours definition
        is_a = [dul.hasTimeInterval]

    class appliesToDay(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [OperatingHours]
        range = [DayOfWeek]
        label = ["applies to day"]
        comment = ["A relation connecting an operating hours entry to a day of the week."]
        # The OperatingHours definition is valid for a specific TimeInterval
        is_a = [dul.associatedWith]

    class hasLanguage(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Tour]
        range = [Language]
        label = ["has language"]
        comment = ["A relation connecting a tour to its language(s)."]
        is_a = [dul.associatedWith]

    # --- Specific Classification Properties ---
    # All these connect an Entity to a Concept (Type), so they are sub-properties of isClassifiedBy
    
    class hasMuseumType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Museum]
        range = [MuseumType]
        label = ["has museum type"]
        comment = ["A relation connecting a museum to its category."]
        is_a = [dul.isClassifiedBy]

    class hasParkType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Park]
        range = [ParkType]
        label = ["has park type"]
        comment = ["A relation connecting a park to its category."]
        is_a = [dul.isClassifiedBy]

    class hasClubType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [NightlifeVenue]
        range = [ClubType]
        label = ["has club type"]
        comment = ["A relation connecting a nightlife venue to its category."]
        is_a = [dul.isClassifiedBy]

    class hasSightType(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Sight]
        range = [SightType]
        label = ["has sight type"]
        comment = ["A relation connecting a sight to its category."]
        is_a = [dul.isClassifiedBy]
    # =========================================================================
    # DATA PROPERTIES
    # =========================================================================
    class opensAt(DataProperty, FunctionalProperty):
        domain = [OperatingHours]
        range = [datetime.time] 
        label = ["opens at"]
        comment = ["An attribute describing the opening time for an Operating Hours entry."]
        is_a = [dul.hasDataValue]

    class closesAt(DataProperty, FunctionalProperty):
        domain = [OperatingHours]
        range = [datetime.time] 
        label = ["closes at"]
        comment = ["An attribute describing the closing time for an Operating Hours entry."]
        is_a = [dul.hasDataValue]

# Save the ontology
onto.save(file="german_city_tourism_final_d2.owl", format="rdfxml")
print('Ontology created and saved')

Ontology created and saved


In [3]:
!pip3 install ontogram

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from owlready2 import *
import subprocess

# Load your ontology from file
onto = get_ontology("german_city_tourism_final_d2.owl").load()

# 1) Save a dot file (class hierarchy graph)
onto.save(file="ontology_graph.dot", format="ntriples")  # we just need the data
# We'll let Graphviz create a visualization from a simple dot wrapper

# Create a simple dot wrapper for classes and subclass edges
def export_class_graph(ontology, dot_path="ontology_classes.dot"):
    with open(dot_path, "w") as f:
        f.write("digraph G {\n")
        for cls in ontology.classes():
            f.write(f'  "{cls.name}" [shape=box];\n')
            for parent in cls.is_a:
                if isinstance(parent, ThingClass):
                    f.write(f'  "{parent.name}" -> "{cls.name}";\n')
        f.write("}\n")

export_class_graph(onto, "ontology_classes.dot")

# 2) Use Graphviz to create a PNG
subprocess.run(["dot", "-Tpng", "ontology_classes.dot", "-o", "ontology_classes.png"], check=True)

print("Generated ontology_classes.png")

Generated ontology_classes.png


In [5]:
# Run the HermiT reasoner to check consistency and compute inferences
from owlready2 import sync_reasoner_hermit

# Sync the reasoner - this will raise an error if the ontology is inconsistent
try:
    with onto:
        sync_reasoner_hermit(infer_property_values=True, ignore_unsupported_datatypes=True)
    print("✅ Ontology is consistent!")
    print(f"Inferred facts have been added to the ontology.")
except OwlReadyInconsistentOntologyError:
    print("❌ Ontology is INCONSISTENT!")

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/os1r1s/.local/lib/python3.10/site-packages/owlready2/hermit:/home/os1r1s/.local/lib/python3.10/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpmiyi_8vb -Y --ignoreUnsupportedDatatypes


✅ Ontology is consistent!
Inferred facts have been added to the ontology.


* Owlready2 * HermiT took 2.2684431076049805 seconds
* Owlready * Equivalenting: owl.Thing DUL.Entity
* Owlready * Equivalenting: DUL.Entity owl.Thing
* Owlready * Reparenting german_tourism_activities.City: {owl.Thing, DUL.PhysicalPlace} => {DUL.PhysicalPlace}
* Owlready * Reparenting german_tourism_activities.MeetingPoint: {owl.Thing, DUL.PhysicalPlace} => {DUL.PhysicalPlace}
* Owlready * Reparenting german_tourism_activities.Duration: {owl.Thing, DUL.Amount} => {DUL.Amount}
* Owlready * Reparenting german_tourism_activities.Language: {owl.Thing, DUL.InformationObject} => {DUL.InformationObject}
* Owlready * Reparenting german_tourism_activities.OperatingHours: {owl.Thing, DUL.TimeInterval} => {DUL.TimeInterval}
* Owlready * Reparenting german_tourism_activities.MuseumType: {owl.Thing, DUL.Concept} => {DUL.Concept}
* Owlready * Reparenting german_tourism_activities.DayOfWeek: {owl.Thing, DUL.Concept} => {DUL.Concept}
* Owlready * Reparenting german_tourism_activities.BudgetTier: {owl

In [6]:
# List all classes and their inferred superclasses
print("=== Classes and their superclasses ===")
for cls in onto.classes():
    print(f"{cls.name}: {[s.name for s in cls.is_a if hasattr(s, 'name')]}")

# Check for any unsatisfiable classes (classes that cannot have instances)
print("\n=== Checking for unsatisfiable classes ===")
unsatisfiable = list(default_world.inconsistent_classes())
if unsatisfiable:
    print(f"❌ Unsatisfiable classes found: {unsatisfiable}")
else:
    print("✅ All classes are satisfiable")

=== Classes and their superclasses ===
Activity: ['Thing', 'Entity']
City: ['PhysicalPlace']
BudgetTier: ['Concept']
LocationSetting: ['Concept']
OperatingHours: ['TimeInterval']
DayOfWeek: ['Concept']
Duration: ['Amount']
Language: ['InformationObject']
MeetingPoint: ['PhysicalPlace']
Tour: ['Activity', 'Event']
PhysicalVenue: ['Activity', 'PhysicalPlace']
Museum: ['PhysicalVenue']
Park: ['PhysicalVenue']
NightlifeVenue: ['PhysicalVenue']
Sight: ['PhysicalVenue']
MuseumType: ['Concept']
ParkType: ['Concept']
ClubType: ['Concept']
SightType: ['Concept']

=== Checking for unsatisfiable classes ===
✅ All classes are satisfiable


In [7]:
# Show inferred class hierarchies (after reasoning)
print("=== Inferred Class Hierarchies ===")
for cls in onto.classes():
    # Get all superclasses (both asserted and inferred)
    all_supers = [s.name for s in cls.is_a if hasattr(s, 'name')]
    # Get only inferred superclasses
    inferred = [s.name for s in cls.INDIRECT_is_a if hasattr(s, 'name') and s not in cls.is_a]
    
    print(f"\n{cls.name}:")
    print(f"  Direct (asserted + inferred): {all_supers}")
    if inferred:
        print(f"  Inferred ancestors: {inferred}")

# Show inferred equivalent classes
print("\n=== Inferred Equivalent Classes ===")
for cls in onto.classes():
    if cls.equivalent_to:
        print(f"{cls.name} ≡ {[e.name for e in cls.equivalent_to if hasattr(e, 'name')]}")

# Show inferred property characteristics
print("\n=== Object Property Inferences ===")
for prop in onto.object_properties():
    print(f"\n{prop.name}:")
    print(f"  Domain: {[d.name for d in prop.domain if hasattr(d, 'name')]}")
    print(f"  Range: {[r.name for r in prop.range if hasattr(r, 'name')]}")
    if hasattr(prop, 'is_a'):
        supers = [s.name for s in prop.is_a if hasattr(s, 'name')]
        if supers:
            print(f"  SubPropertyOf: {supers}")

# Check for any unsatisfiable classes
print("\n=== Checking for unsatisfiable classes ===")
unsatisfiable = list(default_world.inconsistent_classes())
if unsatisfiable:
    print(f"❌ Unsatisfiable classes found: {unsatisfiable}")
else:
    print("✅ All classes are satisfiable")

=== Inferred Class Hierarchies ===

Activity:
  Direct (asserted + inferred): ['Thing', 'Entity']
  Inferred ancestors: ['Activity']

City:
  Direct (asserted + inferred): ['PhysicalPlace']
  Inferred ancestors: ['Thing', 'PhysicalObject', 'Entity', 'City', 'Object']

BudgetTier:
  Direct (asserted + inferred): ['Concept']
  Inferred ancestors: ['Thing', 'BudgetTier', 'SocialObject', 'Entity', 'Object']

LocationSetting:
  Direct (asserted + inferred): ['Concept']
  Inferred ancestors: ['Thing', 'LocationSetting', 'SocialObject', 'Entity', 'Object']

OperatingHours:
  Direct (asserted + inferred): ['TimeInterval']
  Inferred ancestors: ['Abstract', 'Thing', 'OperatingHours', 'Region', 'Entity']

DayOfWeek:
  Direct (asserted + inferred): ['Concept']
  Inferred ancestors: ['Thing', 'SocialObject', 'Entity', 'Object', 'DayOfWeek']

Duration:
  Direct (asserted + inferred): ['Amount']
  Inferred ancestors: ['Abstract', 'Thing', 'Duration', 'Region', 'Entity']

Language:
  Direct (asserted

In [8]:
from graphviz import Digraph
from IPython.display import display, Image

def visualize_with_inferences(ontology, filename="ontology_inferred"):
    dot = Digraph(comment='Ontology with Inferences')
    dot.attr(rankdir='BT', size='20,20', dpi='150', nodesep='0.8', ranksep='1.0')
    
    # Style for nodes
    dot.attr('node', shape='box', style='filled', fillcolor='lightblue', fontsize='12')
    
    for cls in ontology.classes():
        dot.node(cls.name, cls.name)
        
        for parent in cls.is_a:
            if hasattr(parent, 'name') and parent.name != 'Thing':
                # Asserted edges: solid black
                dot.edge(cls.name, parent.name, color='black', arrowhead='empty', penwidth='2')
        
        # Inferred ancestors (indirect) - dashed blue
        for ancestor in cls.INDIRECT_is_a:
            if hasattr(ancestor, 'name') and ancestor not in cls.is_a and ancestor.name != 'Thing':
                dot.edge(cls.name, ancestor.name, color='blue', style='dashed', arrowhead='empty')
    
    dot.render(filename, format='png', cleanup=True)
    return filename + '.png'

img_path = visualize_with_inferences(onto, "german_tourism_inferred")
print(f"Graph with inferences saved to {img_path}")
display(Image(filename=img_path, width=1200))

ModuleNotFoundError: No module named 'graphviz'